In [ ]:
from difflib import SequenceMatcher
from asrtoolkit import cer, wer
import cv2
import pytesseract
import numpy as np
from statistics import mean
import matplotlib.pyplot as plt
from scipy.ndimage import interpolation as inter

In [ ]:

def get_angle(image, delta = 1, center = 0, range = 45):
    def determine_score(arr, angle):
        data = inter.rotate(arr, angle, reshape = False, order = 0)
        histogram = np.sum(data, axis = 1, dtype = object)
        score = np.sum((histogram[1:] - histogram[:-1]) ** 2, dtype = object)
        
        return histogram, score

    scores = []
    angles = np.arange(center - range, center + range + delta, delta)
    for angle in angles:
        histogram, score = determine_score(image, angle)
        scores.append(score)

    best_angle = angles[scores.index(max(scores))]
    return best_angle

def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)

    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    return rotated

def resize_image(image, max_width = 800, max_height = 1000):
    height, width = image.shape[:2]

    scale_with = float(max_width) / width
    scale_height = float(max_height) / height
    
    scale = max(scale_with, scale_height)

    if scale < 1.0:
        return cv2.resize(image, None, fx = scale, fy = scale, interpolation = cv2.INTER_AREA)
    else:
        return image

def correct_skew(image, delta = 0.01, range = 45):
    small = resize_image(image)
    
    _delta = 1
    angle = 0
    while _delta >= delta:
        angle = get_angle(small, _delta, angle, range)
        print(angle)

        range = _delta
        _delta /= 10

    return rotate_image(image, angle)

In [ ]:
def img2text(fileName, process = True):
    img = cv2.imread(fileName, cv2.IMREAD_GRAYSCALE)
    if process:
        img = processImg(img)
        
    return pytesseract.image_to_string(img)

In [ ]:
def processImg(img):
    thres = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 101, 20)
    skew = correct_skew(thres)

    return skew

In [ ]:
def calcCER(begin, end, path = 'test_images/captured/grayscale/..._CC_GRAY.jpg'):
    res = []
    for i in range(begin, end + 1):
        fileName = str(i).zfill(3)
        imageName = path.replace('...', fileName)
        truthName = f'test_images/truth/{fileName}_OCR_ASCII_TEXT_GT.txt'

        text = img2text(imageName)
        with open(truthName, 'r') as f:
            groundTruth = f.read()

        val = cer(groundTruth, text)
        res.append(val)
        print(f'{fileName}: {val}')

    return res

In [ ]:
def imgProcess(begin, end, extension, path = 'test_images/captured/grayscale/..._CC_GRAY.jpg'):
    res = []
    for i in range(begin, end + 1):
        fileName = str(i).zfill(3)
        imageName = path.replace('...', fileName)
        resultName = f'test_images/res/{fileName}{extension}'

        img = cv2.imread(imageName, cv2.IMREAD_GRAYSCALE)
        img = processImg(img)
        cv2.imwrite(resultName, img)

In [ ]:
#write process image
imgProcess(1, 10, 'a.png')

In [ ]:
#calc avg of CER (with pre-process)
CERs = calcCER(12,12)
print(mean(CERs))

In [ ]:
#calc avg of CER (with pre-process) demo image
CERs = calcCER(12, 12, 'demo.png')
print(mean(CERs))

In [ ]:
#calc CER of binary file
CERs = calcCER(11,13, 'test_images/captured/binarized/..._SCANNED_BIN_GT.png')
print(mean(CERs))

In [ ]:
#print text of one image
fileName = '012'
imageName = f'test_images/captured/grayscale/{fileName}_CC_GRAY.jpg'
binaryName = f'test_images/captured/binarized/{fileName}_SCANNED_BIN_GT.png'

print(img2text(binaryName))

In [ ]:
#print text of one demo image
fileName = '012'
imageName = 'res5.png'
binaryName = f'test_images/captured/binarized/{fileName}_SCANNED_BIN_GT.png'

print(img2text(imageName, False))

In [ ]:
#show image after process
fileName = '019'
imageName = f'test_images/captured/grayscale/{fileName}_CC_GRAY.jpg'
img = cv2.imread(imageName, cv2.IMREAD_GRAYSCALE)

img = processImg(img)

cv2.imwrite('_res.png', img)

In [ ]:
#test image process
fileName = 'demo2.jpg'
img = cv2.imread(fileName, cv2.IMREAD_GRAYSCALE)

# https://docs.opencv.org/4.5.2/d7/d4d/tutorial_py_thresholding.html
# _, img = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)
img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 101, 20)

cv2.imwrite('res.jpg', img)